# MEEPART 2D Demo

Welcome to the guide of MEEPART : MEEP for the Analysis of Refractor Telescopes.
It is a step-by-step guide through the main functions of MEEPART in 2D: 
- Obtaining the far field beam of a basic optical system
- Adding various types of defects to the elements of the optical system and studying their effect on the far field beam

We begin by importing basic tools, as numpy, matplotlib for plotting and of course meep_optics.

**This guide is only for the 2D version of MEEPART.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import meep_optics as mo

## Defining the system

### Step 1 : Defining the overall system

*Note that we define the systems with an optical axis along x, the aperture of the telescope being on the left (x=0) and the image plane on the right (x = system size along x).*

First, we define the optical system by giving it a name, and a size : 

In [ ]:
opt_sys = mo.OpticalSystem('Example Telescope')

In the following demonstration, we will be using a system of which we know the physical size. We also know that the system has been extensively studied and performs in the selected range of wavelengths.  

**A note on the size and units:** Maxwell's equations are scale invariant, and this means that MEEP is too. MEEP is made such that c = 1. This means that when specifying dimensions in MEEP (and MEEPART), you have to implicitly choose a length unit to relate to your real units for frequencies and wavelengths studied. There is fortunately a helper function that gives you all the useful information on your system when provided a unit to the meep unit of distance.

Below is an example where we chose the millimeter for the MEEP unit distance, and want to study a 100GHz frequency. The system we want to study is 800mm x 300mm in size. First, we set the size.

In [ ]:
opt_sys.set_size(800,300)

Then, we use the helper function, which returns the real wavelength in meters, the wavelength in MEEP units, the frequency in MEEP units and the corresponding system size.

In [ ]:
opt_sys.sys_info(dist_unit = 1e-3, real_freq = 100e9)

This helps choosing the resolution of the discretization of your system, in points/(meep unit distance). The precision of your computations will increase with resolution. A recommended minimum for wavelength in meep units times resolution is 8. 
For the example above, we then know that choosing a resolution of 4 should get the job done, because wavelength times resolution is 12.

If the required resolution gets to lower than 1 or to high numbers (20+ or more, but this can be extended), you may reconsider your chosen unit of distance (recall that changing it then requires to change the system size, so that your system is always the right physical dimension).

### Step 2 : Defining the lenses
Let's add elements. We begin with two lenses which we know form a focused system.
The parameters to define a lens are : 
- r1 the left surface radius (np.inf means it's a flat surface)
- r2 the right surface radius (np.inf means it's a flat surface)
- c1 the left surface aspheric parameter
- c2 the right surface aspheric parameter
- diameter the diameter of the lens
- thick the thickness of the lens on the optical axis
- x the x position of the intersection of the left surface with the optical axis
- n_refr the index of refraction of the lenses (by default set to HDPE n = 1.52)

The lenses are centered on the optical axis.

The surface equation used here is :
$x = \frac{\frac{y^2}{R}}{1+\sqrt{1-(1+k)\frac{y^2}{R^2}}}$ 
with $x$ the sag from the plane normal to the optical axis, $y$ the vertical distance from the plane, $R$ the surface radius, $k$ the aspheric parameter.

In [ ]:
lens1 = mo.AsphericLens(name = 'Lens 1', 
                     r1 = 327.365, 
                     r2 = np.inf, 
                     c1 = -0.66067, 
                     c2 = 0, 
                     thick = 40,
                     diameter = 300.,
                     x = 140.)

lens2 = mo.AsphericLens(name = 'Lens 2', 
                     r1 = 269.190, 
                     r2 = 6398.02, 
                     c1 = -2.4029, 
                     c2 = 1770.36,
                     thick = 40,
                     diameter = 300.,
                     x = 549.408)
print(lens1)
print(lens2)

### Step 2 : Define the aperture stop
The parameters to define an aperture stop are :
- pos_x the position along the optical axis
- diameter the diameter of the opening
- thickness the thickness of the aperture stop slab
- n_refr the index of refraction of the material
- conductivity the conductivity of the material

Here we chose to make it a metal, with a high conductivity.

In [ ]:
aperture_stop = mo.ApertureStop(name = 'Aperture Stop',
                                 pos_x = 10,
                                 diameter = 200,
                                 thickness = 2,
                                 n_refr = 1.1, 
                                 conductivity = 1e7)
print(aperture_stop)

### Step 3 : Define the image plane
The final element we need is an image plane, defining where the detector(s) will lie. The parameters to define an image plane are :
- pos_x the position along the optical axis
- diameter the size of the image plane along y (orhtogonal to the optical axis)
- thickness the thickness of the slab
- n_refr the index of refraction of the material
- conductivity the conductivity of the material

Here we choose it to be a dielectric

In [ ]:
image_plane = mo.ImagePlane(name = 'Image Plane',
                         pos_x = 10+714.704,
                         diameter = 300,
                         thickness = 2,
                         n_refr = 1.1, 
                         conductivity = 0.01)
print(image_plane)

Now add the components to the optical system :

In [ ]:
opt_sys.add_component(lens1)
opt_sys.add_component(lens2)
opt_sys.add_component(aperture_stop)
opt_sys.add_component(image_plane)

We can check that everything's there :

In [ ]:
opt_sys.list_components()

We know the wavelengths at which our system will operate, let's choose one that is large, allowing us to lower the resolution and thus increase the calculation's speed.

In [ ]:
wavelength = 10

The boundary condition in our study is a Perfectly Matched Layer (a perfect absorber). Its thickness is ideally set to be a half wavelength.

In [ ]:
dpml = 5

We can assemble the system, with an arbitrary resolution (remember that wavelength/resolution should be at least > 8 in the medium with the highest index of refraction) :

In [ ]:
opt_sys.assemble_system(dpml = dpml, res = 2)

Now, we just need to write the h5 file containing the map so that it's later read by the sim.

In [ ]:
opt_sys.write_h5file()

## Running the analysis

Define the simulation from the optical system :

In [ ]:
sim = mo.Sim(opt_sys)
print(sim)

Before anything, let's set the verbosity of MEEP to 0, thus avoiding lengthy prints which are not useful when not debugging.

In [ ]:
sim.set_verbosity(0)

Define the analysis of the optical system :

In [ ]:
analysis = mo.Analysis(sim)

The source is a gaussian source at the image plane. Its beam waist needs to be specified. To decide on that, there is a helper function that converts taper and taper angle to beam waist at the given wavelength. Here are two examples, at a wavelength of 3 meep units, and a taper angle of 5 deg.

In [ ]:
sim.help_gaussian_beam(5, 3, taper = -8)
sim.help_gaussian_beam(5, 3, beam_waist = 10.5)

The analysis is done in the time reverse sense : the electromagnetic field is propagated from a gaussian beam source at the image plane up until the aperture.

We have to provide a runtime, in MEEP units (c=1), the system size is enough to let the EM field propagate from the source to the aperture.

In [ ]:
analysis.image_plane_beams(wvl = wavelength, simres = 2, runtime = 800, beam_w0 = 20.)

We can retrieve the far field beam with the following function, to which we need to provide the diameter of the aperture.

In [ ]:
freq, beam = analysis.beam_FT()

We can plot the system to check the right disposition of the elements and the good propagation of the electric field : 

In [ ]:
%matplotlib inline
analysis.sim.plot_efield()

(Note : if the plot isn't displayed, run the line %matplolib inline again)

We can now plot the far field beam.

In [ ]:
analysis.plotting(freq, beam, wavelength, ylim = -30)

We can add some information to the plot, like the beam solid angle and the best fit gaussian full width at half maximum.

In [ ]:
analysis.plotting(freq, beam, wavelength, ylim=-30, legend = ['Test plot'], print_solid_angle = True, print_fwhm = True)

It is possible to obtain the far field beam for a different sources with different properties, which are wavelength, position on image plane and beam waist. Let's look at different sources at the same time.

In [ ]:
analysis.image_plane_beams(wvl = [10, 12, 8],
                           simres = 1, 
                           runtime = 800,
                           y_source = [0., 50, 100.],
                           beam_w0 = [10, 24, 15])

#freq, beam = analysis.beam_FT()

In [ ]:
analysis.plotting(freq, beam, wavelength, ylim = -30, legend = ['wvl 10', 'wvl 12', 'wvl 8'])

Note that a full simulation is required for each beam, which makes the execution take a long time. This is why it is intersting to put y_max to be the edge of the image plane and N_beams = 2 to obtain only the two extreme situations, beam on axis and beam as far off axis as can be.